# Intro 
In this notebook, we will be looking at an extension to the DQN-based model that we have been using so far. This extension is called Prioritized Experience Replay (PER). Main paper for this repo is [Prioritized Experience Replay](https://arxiv.org/abs/1511.05952).

The general idea is to  sample experiences from the replay buffer in a non-uniform way. Instead of sampling experiences uniformly at random, we will sample experiences with a probability that is proportional to the absolute TD error. This way, we will be more likely to sample experiences that are more surprising or unexpected. This can help the agent learn faster and more effectively.

As discussed previously, TD error is defined as:

$$
\delta = r + \gamma \max_{a'} Q(s', a') - Q(s, a)
$$